In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

I selected a dataset that shows the relationships between comics and heros based on whether the hero is featured in the comic. THe idea was to look at those relationships and see if the comic or the hero was more central, meaning do more heros show up in multiple comics or does the comic relate more frequently to the heroes.

In [20]:

data=pd.read_csv("https://raw.githubusercontent.com/aschwenker/620/master/hero_edges.csv")
comics = list((set(data.comic)))
subset = comics[0:10]
edge_data = data.loc[(data["comic"].isin(subset))]
heros = list((set(edge_data.hero)))

edge_data_list = list(edge_data.itertuples(index=False, name=None))
G = nx.Graph()

G.add_edges_from(edge_data_list)
nodes = heros+subset
data=pd.read_csv("https://raw.githubusercontent.com/aschwenker/620/master/hero_nodes.csv")
node_data = data.loc[(data["node"].isin(nodes))]
#node_data = node_data[["node"]]
node_data_list = list(node_data.itertuples(index=False, name=None))
G.add_nodes_from(node_data_list)

I subset the initial dataset because running on the full set was causing memory errors, to do this I selected 10 of the comics and then selected any node related to that comic, ie the comics and the heroes associated to it

In [21]:
node_data['degcen'] = 0
node_data['eigencen'] = 0
node_data['closecen'] = 0
node_data['betweencen'] = 0
node_data.head()



deg = nx.degree(G) 
eig = nx.eigenvector_centrality(G)
clo = nx.closeness_centrality(G)
btw = nx.betweenness_centrality(G)

for index, row in node_data.iterrows():
    node_data.loc[node_data['node'] == row['node'], 'degcen'] = deg[row['node']]
    node_data.loc[node_data['node'] == row['node'], 'eigencen'] = eig[row['node']]
    node_data.loc[node_data['node'] == row['node'], 'closecen'] = clo[row['node']]
    node_data.loc[node_data['node'] == row['node'], 'betweencen'] = btw[row['node']]

node_data_deg = node_data.groupby("type")['degcen'].mean().reset_index()
node_data_eig = node_data.groupby("type")['eigencen'].mean().reset_index()
node_data_clo = node_data.groupby("type")['closecen'].mean().reset_index()
node_data_btw = node_data.groupby("type")['betweencen'].mean().reset_index()

result = pd.concat([node_data_deg,node_data_eig,node_data_clo,node_data_btw],axis = 1)
result = result.loc[:,~result.columns.duplicated()]
print(result)

C:\Users\aschwenker\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\aschwenker\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\aschwenker\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

    type    degcen  eigencen  closecen  betweencen
0  comic  7.800000  0.124479  0.070470    0.022243
1   hero  1.114286  0.059492  0.070249    0.001713


I found the measures of center and then took the average for each category -- comic & hero. From this we can see the comic category average was much higher, which could be due to the sampling method I chose, in that I first picked comics and then the heroes associated, however, this would not account for the other comics that were related to the heros list, which would make the comics appear artifically more central.